In [1]:
import pandas as pd
import numpy as np
import math
import operator

In [2]:
#lendo dados dos alunos
alunosUFCG = pd.read_csv('../alunosUFCGAnon.csv', thousands = ',')
alunosUFCG.head()

,Matricula,Cod_Curso,Cod_Disciplina,Cod_Evasao,Nome_Curso,Periodo,Nome_Disciplina,Creditos,Departamento,Media_Disciplina,Situacao,Centro,Campus,Creditos_Curso,Tipo,Periodo_Ingresso,Periodo_Relativo
0,B7350,14102100,1411191,0,CIÊNCIA DA COMPUTAÇÃO - D,2004.2,INTERCONEXÃO DE REDES DE COMPUTADORES,2,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,8.5,Aprovado,14,1,208.0,Obrigatória,2002.1,6
1,B7350,14102100,1411196,0,CIÊNCIA DA COMPUTAÇÃO - D,2005.1,INTELIGENCIA ARTIFICIAL I,4,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,9.3,Aprovado,14,1,208.0,Obrigatória,2002.1,7
2,B7350,14102100,1411255,0,CIÊNCIA DA COMPUTAÇÃO - D,2005.1,TECC(LAB DE SISTEMAS OPERACIONAIS),2,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,10.0,Aprovado,14,1,208.0,Optativa,2002.1,7
3,B7350,14102100,1411173,0,CIÊNCIA DA COMPUTAÇÃO - D,2003.2,LÓGICA MATEMÁTICA,4,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,8.7,Aprovado,14,1,208.0,Obrigatória,2002.1,4
4,B7350,14102100,1109035,0,CIÊNCIA DA COMPUTAÇÃO - D,2002.1,ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA,4,UNID. ACAD. DE MATEMÁTICA,9.3,Aprovado,14,1,208.0,Obrigatória,2002.1,1


In [5]:
# lendo dados dos pre requisitos
prerequisitos = pd.read_csv('pre_requisitos.tsv', sep='\t')
prerequisitos.head()

,codigo_curso,curriculo,codigo_disciplina,codigo_prerequisito
0,13301100,1995,1303002,1303001
1,13301100,1995,1301004,1301001
2,13301100,1995,1109084,1109083
3,13301100,1995,1301063,1301041
4,13301100,1995,1301026,1301005


In [6]:
# selecionando os alunos graduados em computação e as colunas desejadas
cadeiras_obrigatorias_df = alunosUFCG[alunosUFCG.Tipo == "Obrigatória"]
cadeiras_obrigatorias_filtradas = cadeiras_obrigatorias_df[['Matricula', 'Cod_Disciplina', 'Nome_Disciplina', 'Periodo', 'Creditos', 'Media_Disciplina', 'Situacao', 'Periodo_Ingresso', 'Periodo_Relativo', 'Nome_Curso']]

In [7]:
# organizando o formato dos dados 
COMPUTACAOdf = cadeiras_obrigatorias_filtradas[cadeiras_obrigatorias_df.Nome_Curso =="CIÊNCIA DA COMPUTAÇÃO - D"]
COMPUTACAOtdf = pd.pivot_table(COMPUTACAOdf, values = 'Media_Disciplina', index = ['Nome_Disciplina'], columns = 'Matricula')
COMPUTACAOdf.head()

,Matricula,Cod_Disciplina,Nome_Disciplina,Periodo,Creditos,Media_Disciplina,Situacao,Periodo_Ingresso,Periodo_Relativo,Nome_Curso
0,B7350,1411191,INTERCONEXÃO DE REDES DE COMPUTADORES,2004.2,2,8.5,Aprovado,2002.1,6,CIÊNCIA DA COMPUTAÇÃO - D
1,B7350,1411196,INTELIGENCIA ARTIFICIAL I,2005.1,4,9.3,Aprovado,2002.1,7,CIÊNCIA DA COMPUTAÇÃO - D
3,B7350,1411173,LÓGICA MATEMÁTICA,2003.2,4,8.7,Aprovado,2002.1,4,CIÊNCIA DA COMPUTAÇÃO - D
4,B7350,1109035,ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA,2002.1,4,9.3,Aprovado,2002.1,1,CIÊNCIA DA COMPUTAÇÃO - D
5,B7350,1411190,REDES DE COMPUTADORES,2004.1,4,8.5,Aprovado,2002.1,5,CIÊNCIA DA COMPUTAÇÃO - D


In [8]:
## Constantes 
# número máximo de vizinhos proximos utilizado
n_vizinhos = 100
# mínimo de similaridade que cada vizinho deve ter com o aluno alvo
pontos_min_viz = 0.7
# número de alunos que se deseja utilizar como vizinhos
n_teste_alunos = 4
# quantidade de cadeiras a serem recomendadas
n_disci_prox_per = 6

In [9]:
## Funções
# checar se dado parâmetro é um número
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))

# cálculo de similaridade entre os alunos
def get_sim(data_frame):
    data_frame.set_index(data_frame.Matricula) 
    inversa_df = data_frame.transpose()
    resultado = inversa_df[is_number(data_frame)].corr(method='pearson')
    return resultado

# eleição dos n_vizinhos mais próximos
def get_neigh(matricula_aluno, df_corr):
    df_corr.at[matricula_aluno, matricula_aluno] = 0
    vizinhos = df_corr[[matricula_aluno,]]

    k_viz_prox = vizinhos.sort_values(by=[matricula_aluno], ascending=False)[1:n_vizinhos+1]
    return k_viz_prox

# cálculo da provável nota do aluno 
# como o numero de vizinhos calculado como otimo foi 3(mt pequeno), é necessário prevenir que muitas pessoas
#fiquem sem predição, logo é preciso aumentar a quantidade de vizinhos para aumentar a chance dele ter pago a cadeira em questao
# mas evitar que a precisão da predição diminua por considerar também vizinhos menos semelhantes
def get_nota(data_frame, k_viz_prox, disc_pred):
    notas_vizinhos = data_frame.loc[list(k_vizinhos.index),]
    notas_vizinhos = notas_vizinhos.loc[notas_vizinhos[disc_pred].notnull()]
        
    #se a quantidade de linhas for ==0, ou seja, não tem vizinhos, não há como calcular a nota
    if len(list(notas_vizinhos.index)) == 0:
        print("Usuario sem vizinhos")
        return -10
    
    notas_vizinhos['Similaridade'] = -0.00

    for index, alunos in notas_vizinhos.iterrows():

        notas_vizinhos.at[index,'Similaridade'] = k_viz_prox.at[index,k_viz_prox.columns[0]]

    notas_validas = notas_vizinhos.loc[notas_vizinhos['Similaridade'] > pontos_min_viz]

    if(len(notas_validas.index) == 0):
        print(disc_pred, ": Sem vizinhos similares")
        return -10

    notas_validas = notas_validas.sort_values(by=['Similaridade'], ascending=False)
    if(len(notas_validas.index) > 4):
        notas_validas = notas_validas[1:4]

    nota_predita = round(sum(notas_validas[disc_pred]*notas_validas['Similaridade'])/ sum(notas_validas['Similaridade']),1)
    return nota_predita

# definição das cadeiras que o aluno pode se matricular a partir do histórico
def prox_cadeiras(historico, sem_prerequisito):
    #seleciona na coluna de prerequisitos as cadeiras que já foram pagas
    possibilidades = prerequisitos.loc[prerequisitos['codigo_prerequisito'].isin(historico)]
    possibilidades = possibilidades[['codigo_disciplina']]
    for cadeira in possibilidades.codigo_disciplina:
        if not cadeira_paga(cadeira,historico):
            prerequisitos_cadeira = prerequisitos.loc[prerequisitos['codigo_disciplina'] == cadeira]

            pre=prerequisitos_cadeira['codigo_prerequisito'].isin(historico)
            prerequisitos_pagos = True
            for  c in pre:
                if c == False:
                    prerequisitos_pagos = False
                    break
            if prerequisitos_pagos:
                sem_prerequisito.append(cadeira)
    return sem_prerequisito

# retorna quais cadeiras o aluno já foi aprovado
def cadeira_paga(cadeira, historico):
    cadeira_isin_historico = False
    for cadeira_paga in historico:
        if cadeira_paga == cadeira:
            cadeira_isin_historico = True
            break
    return cadeira_isin_historico

# retorna o código de uma cadeira a partir do seu nome
def nomeCadeira_para_cod(conj_cadeiras, cod_nome_df):
    cod_conj_cadeiras = []
    for cadeira in conj_cadeiras:
        aux = cod_nome_df.loc[cod_nome_df['Nome_Disciplina'] == cadeira]
        aux = aux[['Cod_Disciplina']]
        if not aux.empty:
            cod_conj_cadeiras.append(aux.iloc[0]['Cod_Disciplina'])
    return cod_conj_cadeiras

# retorna o nome de uma cadeira a partir do seu código
def cod_para_nomeCadeira(conj_cadeiras, cod_nome_df):
    cod_conj_cadeiras = []
    for cadeira in conj_cadeiras:
        aux = cod_nome_df.loc[cod_nome_df['Cod_Disciplina'] == cadeira]
        aux = aux[['Nome_Disciplina']]
    
        if not aux.empty:
            cod_conj_cadeiras.append(aux.iloc[0]['Nome_Disciplina'])
    return cod_conj_cadeiras

In [10]:
# criação do data frame que relaciona cada nome de disciplina com seu respectivo código
cadeiras_cod = COMPUTACAOdf.iloc[:,[1,2]]
cadeiras_code = cadeiras_cod.Cod_Disciplina.unique()
cadeiras_nome = cadeiras_cod.Nome_Disciplina.unique()
nomeCode_cadeiras = pd.DataFrame({"Cod_Disciplina":cadeiras_code, "Nome_Disciplina":cadeiras_nome})
nomeCode_cadeiras.head()

,Cod_Disciplina,Nome_Disciplina
0,1411191,INTERCONEXÃO DE REDES DE COMPUTADORES
1,1411196,INTELIGENCIA ARTIFICIAL I
2,1411173,LÓGICA MATEMÁTICA
3,1109035,ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA
4,1411190,REDES DE COMPUTADORES


In [11]:
# correlação dos alunos
correlacao_alunos = COMPUTACAOtdf.corr()

# organização da representação da matrícula 
alunos_predicao = COMPUTACAOdf
alunos_predicao['Matricula'] = COMPUTACAOdf['Matricula'].map(lambda x: x.lstrip('B'))
alunos_predicao = pd.pivot_table(alunos_predicao, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')

# cadeiras que não precisam de prerrequisito para serem cursadas
sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)']
sem_prerequisito = nomeCadeira_para_cod(sem_prerequisito, nomeCode_cadeiras)
alunos_predicao.head()

/home/liviacavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Nome_Disciplina,ALGEBRA LINEAR I,ANÁLISE E TÉCNICA DE ALGORITMOS,AVAL.DE DESEMPENHO DE SISTEMAS DISCRETOS,BANCO DE DADOS I,BANCO DE DADOS II,CALCULO DIFERENCIAL E INTEGRAL I,CALCULO DIFERENCIAL E INTEGRAL II,COMPILADORES,DIREITO E CIDADANIA,ENGENHARIA DE SOFTWARE I,...,PROJETO EM COMPUTAÇÃO I,PROJETO EM COMPUTAÇÃO II,REDES DE COMPUTADORES,SEMINÁRIOS (EDUCAÇÃO AMBIENTAL),SISTEMAS DE INFORMAÇÃO I,SISTEMAS DE INFORMAÇÃO II,SISTEMAS OPERACIONAIS,TEORIA DA COMPUTAÇÃO,TEORIA DOS GRAFOS,ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA
Matricula,,,,,,,,,,,,,,,,,,,,,
10079,NaN,NaN,NaN,NaN,NaN,5.75,4.95,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,7.8,5.000000
10082,NaN,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1,...,NaN,NaN,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10083,0.0,NaN,NaN,0.0,NaN,5.20,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,7.0,0.0,NaN,7.5,7.1,7.200000
10086,NaN,NaN,NaN,NaN,NaN,1.30,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
10087,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066667


In [12]:
# predição das notas
alunos_predicao = alunos_predicao.sample(n=5)
for index, aluno in alunos_predicao.iterrows():
    # coletando quais cadeiras o aluno já foi aprovado
    cadeiras_pagas = aluno.T.dropna().to_frame()
    cadeiras_pagas = cadeiras_pagas[cadeiras_pagas.iloc[:,0] >= 5]

    cadeira_sem_prerequisito = []

    # vendo quais cadeiras sem prerrequisito já foram pagas pelo aluno
    for cadeira in sem_prerequisito:
        if not cadeira in nomeCadeira_para_cod(list(aluno.index), nomeCode_cadeiras):
            cadeira_sem_prerequisito.append(cadeira)

    # calculando as notas das cadeiras possíveis de serem cursadas
    prox_possiveis_cadeiras = cod_para_nomeCadeira(prox_cadeiras(nomeCadeira_para_cod(list(cadeiras_pagas.index), nomeCode_cadeiras), nomeCadeira_para_cod(cadeira_sem_prerequisito,nomeCode_cadeiras)),nomeCode_cadeiras)
    k_vizinhos = get_neigh('B'+str(index), correlacao_alunos)
    dict_notas = {}
    for cadeira in prox_possiveis_cadeiras:
        nota_aluno = math.trunc(get_nota(COMPUTACAOtdf.T[[cadeira]],k_vizinhos, cadeira)*10)
#         print(nota_aluno)
        if nota_aluno != -100:
            dict_notas[cadeira] = nota_aluno

    # imprimindo notas preditas
    list_notas = sorted(dict_notas.items(), key = operator.itemgetter(1), reverse = True)
    print("Notas do historico do aluno:")
    print(aluno.T.dropna())
    print("-----------")
    print("Notas preditas: ")
    if not list_notas:
        print("Sem recomendaçao")
    else:
        for cadeira in list_notas: 
            print(cadeira[0], ": ", cadeira[1]/10)
    
    print("================ \n")


Notas do historico do aluno:
Nome_Disciplina
ALGEBRA LINEAR I                          1.3
CALCULO DIFERENCIAL E INTEGRAL I          1.2
GERÊNCIA DA INFORMAÇÃO                    0.0
INFORMÁTICA E SOCIEDADE                   7.8
INTRODUÇÃO A COMPUTAÇÃO                   6.1
LABORATÓRIO DE PROGRAMAÇÃO I              7.0
LABORATÓRIO DE PROGRAMAÇÃO II             1.1
LEITURA E PRODUCAO DE TEXTOS              8.3
MATEMÁTICA DISCRETA                       0.0
PROGRAMAÇÃO I                             7.0
PROGRAMAÇÃO II                            0.0
TEORIA DOS GRAFOS                         5.0
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    5.1
Name: 19365, dtype: float64
-----------
Notas preditas: 
LAB.DE ESTRUTURA DE DADOS E ALGORITMOS :  3.4
LABORATÓRIO DE PROGRAMAÇÃO II :  2.0
ALGEBRA LINEAR I :  1.3
PROGRAMAÇÃO II :  0.9

Notas do historico do aluno:
Nome_Disciplina
ALGEBRA LINEAR I                          4.950000
CALCULO DIFERENCIAL E INTEGRAL I          3.800000
CALCULO DIFERENCIAL E I